In [2]:
#Import Libraries:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pickle

In [ ]:
#https://nha.chotot.com/thue-phong-tro-da-nang?page=1
#https://phongtro123.com/tinh-thanh/da-nang

In [ ]:
drive = webdriver.Chrome(ChromeDriverManager().install())
drive.maximize_window()
wait = WebDriverWait(drive, 10)

In [ ]:
page = 1
house_infos = list()
while True:
    try:
        drive.get('https://nha.chotot.com/thue-phong-tro-da-nang?page=' + str(page))
    except: break
    houses = drive.find_elements(By.XPATH, value = '//a[@href and @class="AdItem_adItem__2O28x"]')
    if (houses == []) break;
    for i in range(len(houses)):
        houses[i] = houses[i].get_attribute('href')
    for house in houses:
        drive.get(house)
        try:
            house_location= drive.find_element(By.XPATH, value = '//span[@class="fz13"]').text.replace('Xem bản đồ','')
            house_price = drive.find_element(By.XPATH, value = '//span[@itemprop="price"]').text
            house_name = drive.find_element(By.XPATH, value = '//h1[@class="AdDecription_adTitle__2I0VE" and @itemprop="name"]').text
            house_date = drive.find_element(By.XPATH, value = '//span[@class="AdImage_imageCaptionText__39oDK"]').text
            house_features = drive.find_elements(By.XPATH, value = '//div[@class="col-md-6 no-padding AdParam_adParamItem__1o-dd" and @data-testid="param-item"]')
            house_info = {}
            house_info['Location'] = house_location
            house_info['Price'] = house_price
            house_info['Name'] = house_name
            for index in range(len(house_features)):
                if house_features[index].text[0] == 'T': house_info['Feature' + str(2)] = house_features[index].text
                elif house_features[index].text[0] == 'S': house_info['Feature' + str(3)] = house_features[index].text
                else: house_info['Feature' + str(index)] = house_features[index].text
            house_infos.append(house_info)
        except: continue
    with open('save_page.txt','w') as file:
        file.write(str(page))
        file.close()
    if (page % 10 == 0):
        with open('save_house.pickle','wb') as handle:
            pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    page+=1
with open('save_house.pickle','wb') as handle:
    pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
drive.close()

In [3]:
with open("save_house.pickle","rb") as handle:
    dataset = pickle.load(handle)
dataset = pd.DataFrame(data = dataset)
dataset.head()

,Location,Price,Name,Date,Feature0,Feature1,Feature3,Feature2
0,"17, Cẩm Băc 8, Phường Hòa Thọ Đông, Quận Cẩm L...","1,6 triệu/tháng- 20 m2","Phòng trọ Cẩm Lệ, rộng rãi, sạch đẹp an ninh!",Tin cá nhân đăng 2 tuần trước,Cho thuê,Diện tích: 20 m²,Số tiền cọc: 500.000 đ,NaN
1,"132, Ba Tháng Hai, Phường Thuận Phước, Quận Hả...","1,1 triệu/tháng- 40 m2",Trọ homestay-1.1tr-Điều hoà-Bao điện nước,Tin cá nhân đăng 3 tuần trước,Cho thuê,Diện tích: 40 m²,Số tiền cọc: 50.000 đ,Tình trạng nội thất: Nội thất đầy đủ
2,"K1120/58 Trường Chinh, Phường Hòa Phát, Quận C...","1,5 triệu/tháng- 20 m2",Phòng trọ Quận Cẩm Lệ 20m²,Tin cá nhân đăng 4 tháng trước,Cho thuê,Diện tích: 20 m²,Số tiền cọc: 500.000 đ,Tình trạng nội thất: Nhà trống
3,"Trần Đăng, Phường Hòa Minh, Quận Liên Chiểu, Đ...","2,5 triệu/tháng- 25 m2","Phòng trọ Cao cấp25m² Kinh Dương Vương,Liên Chiểu",Tin cá nhân đăng 1 giờ trước,Cho thuê,Diện tích: 25 m²,Số tiền cọc: 2.500.000 đ,Tình trạng nội thất: Nội thất đầy đủ
4,"Cao Xuân Duc, Phường Thuận Phước, Quận Hải Châ...","1,2 triệu/tháng- 30 m2",trọ homestay,Tin cá nhân đăng 2 giờ trước,Cho thuê,Diện tích: 30 m²,Số tiền cọc: 1.000.000 đ,Tình trạng nội thất: Nội thất đầy đủ


In [ ]:
dataset = pd.DataFrame(data = house_infos)
dataset.head()

In [ ]:
dataset.to_csv('raw_data.csv')